In [10]:
!pip install -U -q tensorflow_io

In [14]:
SEED = 1337
EPOCHS = 100
BATCH_SIZE = 64
VALIDATION_RATIO = 0.1
MODEL_NAME = "uk_irish_accent_recognition"

# Location where the dataset will be downloaded.
# By default (None), keras.utils.get_file will use ~/.keras/ as the CACHE_DIR
CACHE_DIR = None

# The location of the dataset
URL_PATH = "https://www.openslr.org/resources/83/"

# List of datasets compressed files that contain the audio files
zip_files = {
    0: "irish_english_male.zip",
    1: "midlands_english_female.zip",
    2: "midlands_english_male.zip",
    3: "northern_english_female.zip",
    4: "northern_english_male.zip",
    5: "scottish_english_female.zip",
    6: "scottish_english_male.zip",
    7: "southern_english_female.zip",
    8: "southern_english_male.zip",
    9: "welsh_english_female.zip",
    10: "welsh_english_male.zip",
}

# We see that there are 2 compressed files for each accent (except Irish):
# - One for male speakers
# - One for female speakers
# However, we will be using a gender agnostic dataset.

# List of gender agnostic categories
gender_agnostic_categories = [
    "ir",  # Irish
    "mi",  # Midlands
    "no",  # Northern
    "sc",  # Scottish
    "so",  # Southern
    "we",  # Welsh
]

class_names = [
    "Irish",
    "Midlands",
    "Northern",
    "Scottish",
    "Southern",
    "Welsh",
    "Not a speech",
]

In [18]:
import os
import io
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import Audio


# Set all random seeds in order to get reproducible results
keras.utils.set_random_seed(SEED)

# Where to download the dataset
DATASET_DESTINATION = os.path.join(CACHE_DIR if CACHE_DIR else "~/.keras/", "datasets")

In [22]:
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

In [23]:
# CSV file that contains information about the dataset. For each entry, we have:
# - ID
# - wav file name
# - transcript
line_index_file = keras.utils.get_file(
    fname="line_index_file", origin=URL_PATH + "line_index_all.csv"
)

# Download the list of compressed files that contain the audio wav files
for i in zip_files:
    fname = zip_files[i].split(".")[0]
    url = URL_PATH + zip_files[i]

    zip_file = keras.utils.get_file(fname=fname, origin=url, extract=True)
    os.remove(zip_file)

164531638/164531638 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step
103085118/103085118 ━━━━━━━━━━━━━━━━━━━━ 24s 0us/step
166833961/166833961 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
314983063/314983063 ━━━━━━━━━━━━━━━━━━━━ 25s 0us/step
817772034/817772034 ━━━━━━━━━━━━━━━━━━━━ 132s 0us/step
351443880/351443880 ━━━━━━━━━━━━━━━━━━━━ 33s 0us/step
620254118/620254118 ━━━━━━━━━━━━━━━━━━━━ 107s 0us/step
1636701939/1636701939 ━━━━━━━━━━━━━━━━━━━━ 268s 0us/step
1700955740/1700955740 ━━━━━━━━━━━━━━━━━━━━ 354s 0us/step
595683538/595683538 ━━━━━━━━━━━━━━━━━━━━ 125s 0us/step
757645790/757645790 ━━━━━━━━━━━━━━━━━━━━ 84s 0us/step


In [26]:
dataframe = pd.read_csv(
    line_index_file, names=["id", "filename", "transcript"], usecols=["filename"]
)
dataframe.head()

,filename
0,wef_12484_01482829612
1,wef_12484_01345932698
2,wef_12484_00999757777
3,wef_12484_00036278823
4,wef_12484_00458512623


In [28]:
# The purpose of this function is to preprocess the dataframe by applying the following:
# - Cleaning the filename from a leading space
# - Generating a label column that is gender agnostic i.e.
#   welsh english male and welsh english female for example are both labeled as
#   welsh english
# - Add extension .wav to the filename
# - Shuffle samples
def preprocess_dataframe(dataframe):
    # Remove leading space in filename column
    dataframe["filename"] = dataframe.apply(lambda row: row["filename"].strip(), axis=1)

    # Create gender agnostic labels based on the filename first 2 letters
    dataframe["label"] = dataframe.apply(
        lambda row: gender_agnostic_categories.index(row["filename"][:2]), axis=1
    )

    # Add the file path to the name
    dataframe["filename"] = dataframe.apply(
        lambda row: os.path.join(DATASET_DESTINATION, row["filename"] + ".wav"), axis=1
    )

    # Shuffle the samples
    dataframe = dataframe.sample(frac=1, random_state=SEED).reset_index(drop=True)

    return dataframe


dataframe = preprocess_dataframe(dataframe)
dataframe.head()

,filename,label
0,~/.keras/datasets/som_03853_01027933689.wav,4
1,~/.keras/datasets/som_04310_01833253760.wav,4
2,~/.keras/datasets/sof_06136_01210700905.wav,4
3,~/.keras/datasets/som_02484_00261230384.wav,4
4,~/.keras/datasets/nom_06136_00616878975.wav,2


In [36]:
split = int(len(dataframe) * (1 - VALIDATION_RATIO))
train_df = dataframe[:split]
valid_df = dataframe[split:]

print(
    f"We have {train_df.shape[0]} training samples & {valid_df.shape[0]} validation ones")

We have 16089 training samples & 1788 validation ones


In [38]:
@tf.function
def load_16k_audio_wav(filename):
    # Read file content
    file_content = tf.io.read_file(filename)

    # Decode audio wave
    audio_wav, sample_rate = tf.audio.decode_wav(file_content, desired_channels=1)
    audio_wav = tf.squeeze(audio_wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)

    # Resample to 16k
    audio_wav = tfio.audio.resample(audio_wav, rate_in=sample_rate, rate_out=16000)

    return audio_wav


def filepath_to_embeddings(filename, label):
    # Load 16k audio wave
    audio_wav = load_16k_audio_wav(filename)

    # Get audio embeddings & scores.
    # The embeddings are the audio features extracted using transfer learning
    # while scores will be used to identify time slots that are not speech
    # which will then be gathered into a specific new category 'other'
    scores, embeddings, _ = yamnet_model(audio_wav)

    # Number of embeddings in order to know how many times to repeat the label
    embeddings_num = tf.shape(embeddings)[0]
    labels = tf.repeat(label, embeddings_num)

    # Change labels for time-slots that are not speech into a new category 'other'
    labels = tf.where(tf.argmax(scores, axis=1) == 0, label, len(class_names) - 1)

    # Using one-hot in order to use AUC
    return (embeddings, tf.one_hot(labels, len(class_names)))


def dataframe_to_dataset(dataframe, batch_size=64):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["filename"], dataframe["label"])
    )

    dataset = dataset.map(
        lambda x, y: filepath_to_embeddings(x, y),
        num_parallel_calls=tf.data.experimental.AUTOTUNE,
    ).unbatch()

    return dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)


train_ds = dataframe_to_dataset(train_df)
valid_ds = dataframe_to_dataset(valid_df)

AttributeError: in user code:

    File "/var/folders/_4/yms79tz157x7y0wy5lmykw2m0000gn/T/ipykernel_77287/3678912734.py", line 44, in None  *
        lambda x, y: filepath_to_embeddings(x, y)
    File "/var/folders/_4/yms79tz157x7y0wy5lmykw2m0000gn/T/ipykernel_77287/3678912734.py", line 19, in filepath_to_embeddings  *
        audio_wav = load_16k_audio_wav(filename)
    File "/var/folders/_4/yms79tz157x7y0wy5lmykw2m0000gn/T/ipykernel_77287/3678912734.py", line 12, in load_16k_audio_wav  *
        audio_wav = tfio.audio.resample(audio_wav, rate_in=sample_rate, rate_out=16000)
    File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow_io/python/ops/audio_ops.py", line 466, in f  *
        i, rate_in=rate_in, rate_out=rate_out, name=name
    File "/opt/anaconda3/lib/python3.11/site-packages/tensorflow_io/python/ops/__init__.py", line 88, in __getattr__
        return getattr(self._load(), attrb)

    AttributeError: module '77ab628d7ad4acaa62f6bde524b9d631895821c9' has no attribute

In [40]:
tfio.audio.resample

Type:        module
String form: <module 'tensorflow_io' from '/opt/anaconda3/lib/python3.11/site-packages/tensorflow_io/__init__.py'>
File:        /opt/anaconda3/lib/python3.11/site-packages/tensorflow_io/__init__.py
Docstring:   tensorflow_io